# DS211 : Assigment 2 Submission

## Answer 2

We first define $r$ which is the cash requirement for $n=8$ quarters. We also define variables $p$, $f$, $g$, $h$ which are the factors to be multiplied to the principal to calculate the amount of a loan or transaction.

In [42]:
from scipy.optimize import linprog
import numpy as np

# Cash requirement for each quarter
r = 100*np.array([1, 5, 1, -6, -5, 2, 6, -9])
n = r.shape[0] # Quarters
p = 1.01**n    # 1% 8 Quarter
f = 1.018**2   # 1.8% 2 Quarter
g = 1.025      # 2.5% 1 Quarter
h = 1.005      # 0.5% 1 Quarter

Let $w$ be the initial two year loan,
$x_i$ be the half yearly loan taken in the $i$-th quarter,
$y_i$ be the quarterly loan taken in the $i$-th quarter
and
$z_i$ be the quarterly investment for $i$-th quarter.

Now, the constraints from the cash requirments are as follows
$$
w + x_1 + y_1 - z_1 = b_1 \\
x_2 + y_2 - z_2  - g y_1 + h z_1 = b_2 \\
x_3 + y_3 - z_3  - f x_1 - g y_2 + h z_2 = b_3 \\
... \\
x_8 + y_8 - z_8  - f x_6 - g y_7 + h z_7 = b_3 \\
$$

To maximize the wealth in the end we need to minimize the following cost function
$$
p w + f x_7 + f x_8 + g y_8 - h z_8
$$

We use `scipy.linprog` to solve the LP problem and draw the following inferences.

In [43]:
b = r                         # From the table
A = np.zeros((n+2,3*n+1))     # 2 extra quarter calc
A[0,0] = 1                    # Initial loan w
A[n,0] = -p                   # Repayment    
for i in range(n):
    #  2 quarter loan
    A[i,3*i+1] = 1
    A[i+2,3*i+1] = -f
    # 1 quarter loan
    A[i,3*i+2] =  1
    A[i+1,3*i+2] = -g
    # 1 quarter investment
    A[i,3*i+3] =  -1
    A[i+1,3*i+3] =  h
c = -A[n:,:].sum(axis=0)      # Cost function cTx
A = A[:n,:]

res = linprog(c, A_eq = A, b_eq = b)

print(f"Initial loan for 2 years : {res.x[0]} crore INR")
print(f"Half yearly loan to be taken is\n{res.x[1::3]} crore INR")
print(f"Quarterly loan to be taken is\n{res.x[2::3]} crore INR")
print(f"The cost function value is {c.dot(res.x)} crore INR")

Initial loan for 2 years : 399.8099217133485 crore INR
Half yearly loan to be taken is
[  0.         198.69102868   0.           0.           0.
   0.           0.           0.        ] crore INR
Quarterly loan to be taken is
[  0.   0. 100.   0.   0.   0.   0.   0.] crore INR
The cost function value is -471.56314529606266 crore INR


The following is an implementation of the revised simplex method and phase1 program to find and initial feasible point given the constraints $Ax = b$.

In [44]:
def simplex(c, A, b, x):
    # Assuming n > m and that A is full rank and x is feasible
    n = len(c) # Number of variables 
    m = len(b) # Number of constraints 
    
    nbasic = np.where(np.absolute(x) < 1e-5)[0][:n-m]
    basic = np.setdiff1d(np.arange(n), nbasic)

    for k in range(100):
        B = A[:, basic]  
        print(B.shape,  np.linalg.matrix_rank(B))
        N = A[:, nbasic] 
        xB = x[basic]
        cB = c[basic]
        cN = c[nbasic]
        la = np.linalg.solve(B.T, cB)
        sN = cN - N.T @ la
        if all(sN >= 0): # optimal point found 
            print(f"Optimal found")
            return x
        min_sN = np.argmin(sN)
        q = nbasic[min_sN]
        d = np.linalg.solve(B, A[:, q])
        if all(d < 0):
            print("Problem unbounded")
            return
        i_dpos = np.where(d > 0)[0]
        fract = xB[i_dpos] / d[i_dpos]
        xqpos = np.min(fract)
        p = i_dpos[np.argmin(fract)] 
        indq, = np.where(nbasic == q)
        xB_plus = xB - d * xqpos
        xN_plus = np.zeros(n - m)
        xN_plus[indq] = xqpos
        x[basic] = xB_plus
        x[nbasic] = xN_plus
        nbasic[indq] = basic[p]
        basic[p] = q

    print("Could not converge")
    return

In [45]:
def phase1(A,b):
    m, n = A.shape
    e = np.concatenate((np.zeros(n), np.ones(m)))
    E = np.diag(-1 + 2 * (b >= 0))
    A_p1 = np.hstack((A, E))
    x = np.concatenate((np.zeros(n), np.abs(b)))
    x = simplex(e, A_p1, b, x)
    x = x[:n]
    return x

In [46]:
x = phase1(A,b)
ans = simplex(c, A, b, x)

(8, 8) 8
(8, 8) 8
(8, 8) 8
(8, 8) 8
(8, 8) 8
(8, 8) 8
(8, 8) 8
(8, 8) 8
(8, 8) 8
Optimal found
(8, 8) 8
(8, 8) 8
(8, 8) 8
(8, 8) 8
(8, 8) 8
Optimal found


We get the same solution again using this implementation

In [47]:
print(f"Initial loan for 2 years : {ans[0]} crore INR")
print(f"Half yearly loan to be taken is\n{ans[1::3]} crore INR")
print(f"Quarterly loan to be taken is\n{ans[2::3]} crore INR")
print(f"The cost function value is {c.dot(ans)} crore INR")

Initial loan for 2 years : 399.8099217133486 crore INR
Half yearly loan to be taken is
[  0.         198.69102868   0.           0.           0.
   0.           0.           0.        ] crore INR
Quarterly loan to be taken is
[  0.   0. 100.   0.   0.   0.   0.   0.] crore INR
The cost function value is -471.56314529606254 crore INR


Now we use the different options of `scipy.linprog` to solve the LP problem. Interestingly, we get a deprecation warning in doing so but find the solutions to be close to the order of $10^{-6}$.

In [48]:
res1 = linprog(c, A_eq = A, b_eq = b, method="interior-point")
res2 = linprog(c, A_eq = A, b_eq = b, method="revised simplex")

/tmp/ipykernel_32885/1516641352.py:1: DeprecationWarning: `method='interior-point'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res1 = linprog(c, A_eq = A, b_eq = b, method="interior-point")
/tmp/ipykernel_32885/1516641352.py:2: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res2 = linprog(c, A_eq = A, b_eq = b, method="revised simplex")


In [49]:
print(f"The difference between the two solutions : {res1.x - res2.x}")

The difference between the two solutions : [ 4.76691463e-06  1.32057270e-06  1.40478013e-06  7.52365850e-06
  3.56094020e-06  1.52822621e-06  1.13703662e-05  3.17177011e-06
 -1.03103099e-05  1.38571710e-06  8.77181363e-07  1.10948347e-06
  1.00650314e-05  2.11517838e-06  1.16836264e-06  8.81450558e-06
  1.62844622e-06  1.03894593e-06  9.48343188e-06  1.35902484e-06
  7.30899386e-08  7.89817638e-06  9.24827220e-07  1.56374295e-06
  8.37556183e-06]


Finally we list out the cost function values for the 4 ways we have solved the LP problem.

In [50]:
print(f"linprog : {c.dot(res.x)}")
print(f"self implemented simplex : {c.dot(ans)}")
print(f"linprog interior point: {c.dot(res1.x)}")
print(f"linprog revised simplex: {c.dot(res2.x)}")

linprog : -471.56314529606266
self implemented simplex : -471.56314529606254
linprog interior point: -471.5631445819696
linprog revised simplex: -471.5631452960626
